In [6]:
import rioxarray as rioxr
from dataclasses import asdict
from dscreator.cfarray.attributes import DatasetAttrsGrid, VariableAttrs, LongitudeAttrs, LatitudeAttrs
from dscreator import utils
import xarray as xr
import os
from dscreator.storage import get_storage_handler
from dscreator.config import SETTINGS

urchin_da = rioxr.open_rasterio(
    "S_droebachiensis_final_biomass_sqm_wgs84.tif",
    masked=False
).drop_vars("band")

name = "Predicted_urchin_biomass"
urchin_da = urchin_da.squeeze("band", drop=True)
urchin_da.name = name
urchin_da.attrs = asdict(VariableAttrs(long_name="Strongylocentrotus droebachiensis predicted biomass", units="g m-2", short_name=name))


In [7]:
urchin_da.x.attrs = asdict(LongitudeAttrs())
urchin_da.y.attrs = asdict(LatitudeAttrs())
urchin_da = urchin_da.drop_vars("spatial_ref")

In [8]:
ds = xr.merge([urchin_da])
ds.attrs = asdict(
    DatasetAttrsGrid(
        title="Strongylocentrotus droebachiensis predicted biomass (g/m2)",
        title_no="Strongylocentrotus droebachiensis predicted biomass (g/m2)",
        summary="Predicted biomass of Strongylocentrotus droebachiensis from the combined products of a geospatial Random Forest model and an ensemble Specis Distribution Model, from the manuscript: (Insert link here when available).",
        summary_no="Strongylocentrotus droebachiensis predicted biomass (g/m2)",
        creator_email="charles.lavin@niva.no",
        creator_name="Charles P. Lavin",
        creator_institution="Norsk institutt for vannforskning",
        creator_type="person",
        # https://gcmd.earthdata.nasa.gov/KeywordViewer
        keywords=",".join(
            [
                "GCMDSK:Science Keywords > EARTH SCIENCE > BIOLOGICAL CLASSIFICATION > ANIMALS/INVERTEBRATES > ECHINODERMS > SEA URCHINS",
                "GCMDLOC:CONTINENT > EUROPE > NORTHERN EUROPE > SCANDINAVIA > NORWAY",
            ]
        ),
        keywords_vocabulary=",".join(
            [
                "GCMDSK:GCMD Science Keywords:https://gcmd.earthdata.nasa.gov/kms/concepts/concept_scheme/sciencekeywords",
                "GCMDLOC:GCMD Locations:https://gcmd.earthdata.nasa.gov/kms/concepts/concept_scheme/locations",
            ]
        ),
        time_coverage_end="",
        time_coverage_start="",
        iso_topic_category="environment",
        spatial_representation="grid",
        date_created="2025-12-01T12:00:00Z",
        project="230189 - Removal and utilization of sea urchin barrens to restore macroalgae forests, eco",
        geospatial_lon_min=float(ds.x.min()),
        geospatial_lon_max=float(ds.x.max()),
        geospatial_lat_min=float(ds.y.min()),
        geospatial_lat_max=float(ds.y.max()),
        history=f"",
    )
)
ds.attrs["id"] = "522d5685-7157-407c-bb22-6a882ed5cbd8"

In [ ]:
SETTINGS.storage_path = os.path.join(os.getcwd(), "..", "catalog")
#SETTINGS.storage_path = "gs://nivaprod-1-senda"

ds['x'] = ds['x'].astype('float32')
ds['y'] = ds['y'].astype('float32')
# For transfer directly into the test thredds server use

get_storage_handler(
    grouping="refarktis",
    dataset_name="S_droebachiensis_final_biomass_sqm_wgs84",
    filename_prefix="area-density").save_dataset(ds)

'gs://nivaprod-1-senda/datasets/refarktis/S_droebachiensis_final_biomass_sqm_wgs84/area-density_acdd_s_droebachiensis_final_biomass_sqm_wgs84.nc'